In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
import random

## Отложим тест

In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,text,label
0,71977,"а, ну надо перед циклом i = 0 написать",0
1,61004,Прикол в принципиальном отличии алгоритма\nНап...,2
2,27119,"Кстати, я так понял тема-то устройства интерес...",2
3,96329,ДЕВЧЕНКИ ВРЕМЯ НЕ ТЕРЯЙТЕ ном.НАБИРАЙТЕ: (063...,1
4,4707,п29 хочу *. Отношения! Ответ на ммс с Дев. 068...,1


In [3]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=123456)
print(train_df.shape, test_df.shape)

(72000, 3) (8000, 3)


In [4]:
train_df.to_csv('my_train.csv', index=False)
test_df.to_csv('my_test.csv', index=False)

# Эксперименты с моделями 

## Обучим модель для классификации

In [5]:
train_df = pd.read_csv('my_train.csv')
test_df = pd.read_csv('my_test.csv')
train_df.head()

,id,text,label
0,11107,П.29л позн.с девушкой для общения с/о.Днепр.06...,1
1,110307,вывод - надо было не брать ноуты... был бы тож...,2
2,101112,может версия зип-архива более новая конечно,0
3,88170,"но в целом - да, для разных языков надо разные...",0
4,78794,.МУЖ.би.СИМПТ.ДЛЯ.Д/Ж.К..ИГ.МАССАЖ.0984990115.,1


In [6]:
# фиксируем все рандом сиды, чтобы получать такой же результат при запуске train, predict
my_seed = 12345
np.random.seed(my_seed)
random.seed(my_seed)

In [8]:
tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(train_df['text'])
X_test = tfidf.transform(test_df['text'])
print(X_train.shape, X_test.shape)

y_train, y_test = train_df['label'].values, test_df['label'].values

(72000, 94464) (8000, 94464)


In [9]:
clf = SGDClassifier()
clf.fit(X_train, y_train)
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [10]:
f_train = f1_score(y_train, y_train_pred, average='micro')
f_test = f1_score(y_test, y_test_pred, average='micro')
f_train, f_test

(0.8576944444444444, 0.8435)

# Код для сдачи

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
import pickle

CLF_PATH = 'clf.pkl'
TFIDF_PATH = 'tfidf.pkl'
SUBMIT_PATH = 'submit.csv'

def train(train_path):
    my_seed = 12345
    np.random.seed(my_seed)
    random.seed(my_seed)

    df = pd.read_csv(train_path)
    tfidf = TfidfVectorizer()
    X = tfidf.fit_transform(df['text'])
    y = df['label'].values
    clf = SGDClassifier()
    clf.fit(X, y)
    pickle.dump(clf, open(CLF_PATH, 'wb'))
    pickle.dump(tfidf, open(TFIDF_PATH, 'wb'))

def predict(test_path):
    df = pd.read_csv(test_path)
    tfidf = pickle.load(open(TFIDF_PATH, 'rb'))
    clf = pickle.load(open(CLF_PATH, 'rb'))
    X = tfidf.transform(df['text'])
    y = clf.predict(X)
    df['label'] = y
    df.to_csv(SUBMIT_PATH)

# Так можно проверить, что с кодом для сдачи все хорошо

In [13]:
train('my_train.csv')
predict('my_test.csv')
y_pred = pd.read_csv('submit.csv')['label'].values
y_test =  pd.read_csv('my_test.csv')['label'].values
f1_score(y_test, y_pred, average='micro')

0.8435